<a href="https://colab.research.google.com/github/rbkellam/2019-Fantasy-Football-data/blob/main/LIT_Take_Home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import statistics as stats
import io



In [ ]:
from google.colab import files
uploaded = files.upload()
# selecting downloaded RWTCd and 3MO (1) files that I downloaded from email

Saving RWTCd.xls to RWTCd (2).xls
Saving 3MO_(1).csv to 3MO_(1) (2).csv


In [ ]:
oil = pd.read_excel(io.BytesIO(uploaded['RWTCd.xls']), sheet_name='Data 1')
treasury = pd.read_csv(io.StringIO(uploaded['3MO_(1).csv'].decode('utf-8')))

In [ ]:
oil.head()

,Back to Contents,"Data 1: Cushing, OK WTI Spot Price FOB (Dollars per Barrel)"
0,Sourcekey,RWTC
1,Date,"Cushing, OK WTI Spot Price FOB (Dollars per Ba..."
2,1986-01-02 00:00:00,25.56
3,1986-01-03 00:00:00,26
4,1986-01-06 00:00:00,26.53


In [ ]:
blank = treasury[treasury['DTB3'] == '.' ].index
treasury.drop(blank, inplace=True)
# removing holidays or missing data from treasury dataset
treasury.head(20)

,DATE,DTB3
0,2016-08-26,0.33
1,2016-08-29,0.33
2,2016-08-30,0.33
3,2016-08-31,0.33
4,2016-09-01,0.33
5,2016-09-02,0.32
7,2016-09-06,0.32
8,2016-09-07,0.34
9,2016-09-08,0.34
10,2016-09-09,0.34


In [ ]:
oil.rename(columns = {"Data 1: Cushing, OK WTI Spot Price FOB (Dollars per Barrel)":"Price"}, inplace="True")
oil.rename(columns = {"Back to Contents":"Date"}, inplace="True")
oil.drop(oil.index[:2], inplace=True) # first two rows in oil unrelated to dataset
oil['Date'] = pd.to_datetime(oil['Date']).dt.date
treasury.rename(columns = {"DATE":"Date"}, inplace="True") 
combined_df.dropna(axis=0, inplace=True)
oil.Date = oil.Date.astype('str') # needs to be in string form in order for join to work in this instance I believe
oil.head()

,Date,Price
8,1986-01-10,25.65
9,1986-01-13,25.08
10,1986-01-14,24.97
11,1986-01-15,25.18
12,1986-01-16,23.98


In [ ]:
combined_df = oil.join(treasury.set_index('Date'), on='Date', how='inner')
combined_df["Price"] = pd.to_numeric(combined_df["Price"], downcast="float")
combined_df["DTB3"] = pd.to_numeric(combined_df["DTB3"], downcast="float")
pd.options.display.float_format = "{:,.2f}".format
# transform both from object to float so I can do calculations later
print(combined_df)

            Date  Price  DTB3
7734  2016-08-26  47.64  0.33
7735  2016-08-29  46.97  0.33
7736  2016-08-30  46.32  0.33
7737  2016-08-31  44.68  0.33
7738  2016-09-01  43.17  0.33
...          ...    ...   ...
8942  2021-06-24  73.31  0.05
8943  2021-06-25  74.21  0.06
8944  2021-06-28  72.98  0.05
8945  2021-06-29  73.14  0.04
8946  2021-06-30  73.52  0.05

[1203 rows x 3 columns]


In [ ]:
combined_df['Date'] = pd.to_datetime(combined_df['Date'], format='%Y-%m-%d')
# transform string to datetime object so I can filter
filtered_df = combined_df.loc[(combined_df['Date'] >= '2019-12-31') & (combined_df['Date'] <= '2020-12-31')]
print(filtered_df)

           Date  Price  DTB3
8570 2019-12-31  61.14  1.52
8571 2020-01-02  61.17  1.51
8572 2020-01-03  63.00  1.49
8573 2020-01-06  63.27  1.53
8574 2020-01-07  62.70  1.51
...         ...    ...   ...
8818 2020-12-24  48.18  0.09
8819 2020-12-28  47.50  0.11
8820 2020-12-29  47.85  0.10
8821 2020-12-30  48.24  0.08
8822 2020-12-31  48.35  0.09

[250 rows x 3 columns]


In [ ]:
import warnings
warnings.filterwarnings('ignore')

filtered_df['oil_rets'] = filtered_df['Price'].pct_change()
filtered_df['tbill_rets'] = filtered_df['DTB3'].pct_change()
filtered_df['simple_excess_rets'] = oil_rets - tbill_rets
# This answers part 1 of take-home test
print(filtered_df)

           Date  Price  DTB3  oil_rets  tbill_rets  simple_excess_rets
8570 2019-12-31  61.14  1.52       nan         nan                 nan
8571 2020-01-02  61.17  1.51      0.00       -0.01                0.01
8572 2020-01-03  63.00  1.49      0.03       -0.01                0.04
8573 2020-01-06  63.27  1.53      0.00        0.03               -0.02
8574 2020-01-07  62.70  1.51     -0.01       -0.01                0.00
...         ...    ...   ...       ...         ...                 ...
8818 2020-12-24  48.18  0.09      0.01        0.00                0.01
8819 2020-12-28  47.50  0.11     -0.01        0.22               -0.24
8820 2020-12-29  47.85  0.10      0.01       -0.09                0.10
8821 2020-12-30  48.24  0.08      0.01       -0.20                0.21
8822 2020-12-31  48.35  0.09      0.00        0.13               -0.12

[250 rows x 6 columns]


In [ ]:
cum_return_oil = (filtered_df['oil_rets'].iloc[-1] - filtered_df['oil_rets'].iloc[1]) / filtered_df['oil_rets'].iloc[1]
cum_return_oil
# for this cell and next, you do not add up simple averages (you multiply) but just curious what the sum of percent changes was
#iloc [-1] avoid the NaN row

3.6472304

In [ ]:
cum_return_tbill = (filtered_df['tbill_rets'].iloc[-1] - filtered_df['tbill_rets'].iloc[1]) / filtered_df['tbill_rets'].iloc[1]
cum_return_tbill

-20.000092

In [ ]:
log_data = [filtered_df["Date"], filtered_df["Price"], filtered_df["DTB3"]]
log_headers = ["Date", "Price", "DTB3"]
log_df = pd.concat(log_data, axis=1, keys=log_headers)
# creating new dataframe connected to filtered_df so I can calculate separate log returns
log_df['oil_rets'] = np.log(log_df['Price']/log_df['Price'].shift(1))
log_df['tbill_rets'] = np.log(log_df['DTB3']/log_df['DTB3'].shift(1))
log_df['excess_rets'] = log_df['oil_rets'] - log_df['tbill_rets']
print(log_df)

           Date  Price  DTB3  oil_rets  tbill_rets  excess_rets
8570 2019-12-31  61.14  1.52       nan         nan          nan
8571 2020-01-02  61.17  1.51      0.00       -0.01         0.01
8572 2020-01-03  63.00  1.49      0.03       -0.01         0.04
8573 2020-01-06  63.27  1.53      0.00        0.03        -0.02
8574 2020-01-07  62.70  1.51     -0.01       -0.01         0.00
...         ...    ...   ...       ...         ...          ...
8818 2020-12-24  48.18  0.09      0.00        0.00         0.00
8819 2020-12-28  47.50  0.11     -0.01        0.20        -0.21
8820 2020-12-29  47.85  0.10      0.01       -0.10         0.10
8821 2020-12-30  48.24  0.08      0.01       -0.22         0.23
8822 2020-12-31  48.35  0.09      0.00        0.12        -0.12

[250 rows x 6 columns]


In [ ]:
array = log_df.iloc[:,-1].values
array= np.around(array, decimals=2, out=None)
# for finding cumulative return in next step, I found it easier to transform the column to numpy array first
print(array)

[  nan  0.01  0.04 -0.02  0.   -0.05 -0.   -0.01 -0.03  0.   -0.01  0.02
 -0.01 -0.01 -0.02 -0.02 -0.02 -0.03 -0.01  0.01 -0.03  0.   -0.04 -0.01
  0.03  0.   -0.01 -0.03  0.01  0.02 -0.    0.02  0.    0.02  0.01  0.01
 -0.02 -0.03 -0.02  0.02  0.08  0.17  0.18  0.26  0.13  0.19  0.06 -0.19
  0.01  0.17  0.21  0.03  0.16  1.98 -0.49 -0.48  1.1   0.59   nan -0.45
   nan -1.48  0.46  0.19  0.22  0.01 -0.48 -0.04 -0.39 -0.21 -0.06  0.16
  0.35 -0.01  0.07   nan   nan  0.34  0.19 -0.03 -0.27  0.11  0.29  0.35
 -0.26 -0.04  0.18  0.05  0.08 -0.04 -0.03 -0.01 -0.02  0.16  0.07 -0.
  0.01  0.12  0.02 -0.02 -0.12 -0.13  0.03  0.12 -0.   -0.03 -0.05  0.07
  0.05 -0.16 -0.09  0.13 -0.08  0.06 -0.1   0.09 -0.01  0.08  0.09 -0.04
 -0.    0.   -0.04  0.13  0.03 -0.14  0.15  0.02 -0.07  0.    0.01  0.11
  0.02 -0.09 -0.06 -0.04  0.36 -0.   -0.16  0.02  0.    0.06  0.09  0.01
 -0.01  0.01  0.17  0.01 -0.09  0.13 -0.09 -0.01 -0.02 -0.08 -0.01  0.03
  0.09 -0.    0.02  0.11 -0.2  -0.01  0.09 -0.18  0.1

In [ ]:
total = np.nansum(array)
# total refers to cumulative log excess return in decimal form
# This answers part 2 of take-home test
# log cumulative excess return is 423%
print(total)

4.2300005
